In [105]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

####################  Data Analysis & Calculation  #####################3
import numpy as np   
import pandas as pd  
import datetime  

####################  Visuvalization & plotting  #####################3
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline  
#################### Machine Learning #####################3
from sklearn.ensemble import AdaBoostClassifier 
from sklearn.ensemble import AdaBoostRegressor  

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
import warnings  
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder 

In [107]:
df=pd.read_csv(r"C:\Users\91909\Desktop\PROJECTS\baseball regression project\baseball.csv")
df.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
1,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A
2,496,141,20,65,78,37,11,5628,1575,225,828,838,354,N,E,200,11,3,500.0,N
3,321,87,10,39,42,30,2,396,101,12,48,46,33,N,E,805,40,4,91.5,N
4,594,169,4,74,51,35,11,4408,1133,19,501,336,194,A,W,282,421,25,750.0,A


In [109]:
df.NewLeague.value_counts()

NewLeague
A    141
N    122
Name: count, dtype: int64

In [111]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 263 entries, 0 to 262
Data columns (total 20 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   AtBat      263 non-null    int64  
 1   Hits       263 non-null    int64  
 2   HmRun      263 non-null    int64  
 3   Runs       263 non-null    int64  
 4   RBI        263 non-null    int64  
 5   Walks      263 non-null    int64  
 6   Years      263 non-null    int64  
 7   CAtBat     263 non-null    int64  
 8   CHits      263 non-null    int64  
 9   CHmRun     263 non-null    int64  
 10  CRuns      263 non-null    int64  
 11  CRBI       263 non-null    int64  
 12  CWalks     263 non-null    int64  
 13  League     263 non-null    object 
 14  Division   263 non-null    object 
 15  PutOuts    263 non-null    int64  
 16  Assists    263 non-null    int64  
 17  Errors     263 non-null    int64  
 18  Salary     263 non-null    float64
 19  NewLeague  263 non-null    object 
dtypes: float64

In [113]:
sns.histplot(df['Salary'])

<Axes: xlabel='Salary', ylabel='Count'>

### right skewed target variable , needs to be transformed (log on y)

In [116]:
# df['Salary']=df['Salary'].apply(lambda x:np.log(x))

In [118]:
df.Salary = np.log(df.Salary)


In [120]:
sns.histplot(df['Salary'])

<Axes: xlabel='Salary', ylabel='Count'>

In [122]:
sns.boxplot(df)

<Axes: xlabel='Salary', ylabel='Count'>

In [123]:
for i in df:                  # i = column name 
    if df[i].dtypes in ("float64","int64"): # df[i] = every column will select float & int column only
        q1 = df[i].quantile(0.25)  # for 1 column compute q1
        q3 = df[i].quantile(0.75)  # for 1 column compute q3
        iqr = q3-q1  # for 1 column compute IQR
        ul = q3 + 1.5*iqr # for 1 column compute UPPER LIMIT
        ll = q1 - 1.5*iqr # for 1 column compute LOWER LIMIT 
        df[i].clip(lower=ll,upper=ul,inplace=True)

sns.boxplot(df)

<Axes: xlabel='Salary', ylabel='Count'>

In [126]:
# #Employee Numeric columns
df_num = df.select_dtypes(include=[np.number])
df_num.head(3)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary
0,315,81,7.0,24,38,39,14,3449.0,835,69.00,321.0,414.0,375.0,632,43,10,6.163315
1,479,130,18.0,66,72,76,3,1624.0,457,63.00,224.0,266.0,263.0,636,82,14,6.173786
2,496,141,20.0,65,78,37,11,5628.0,1575,208.75,828.0,838.0,354.0,200,11,3,6.214608


In [128]:
df_cat = df.select_dtypes(include=['object'])
df_cat.head(3)

,League,Division,NewLeague
0,N,W,N
1,A,W,A
2,N,E,N


In [130]:
from sklearn.preprocessing import LabelEncoder

df_cat = df_cat.apply(LabelEncoder().fit_transform)
df_cat.head(3)


,League,Division,NewLeague
0,1,1,1
1,0,1,0
2,1,0,1


In [132]:
df_final = pd.concat([df_num,df_cat],axis=1)
df_final.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,Salary,League,Division,NewLeague
0,315,81,7.0,24,38,39,14,3449.0,835,69.00,321.0,414.0,375.0,632,43,10,6.163315,1,1,1
1,479,130,18.0,66,72,76,3,1624.0,457,63.00,224.0,266.0,263.0,636,82,14,6.173786,0,1,0
2,496,141,20.0,65,78,37,11,5628.0,1575,208.75,828.0,838.0,354.0,200,11,3,6.214608,1,0,1
3,321,87,10.0,39,42,30,2,396.0,101,12.00,48.0,46.0,33.0,636,40,4,4.516339,1,0,1
4,594,169,4.0,74,51,35,11,4408.0,1133,19.00,501.0,336.0,194.0,282,421,25,6.620073,0,1,0


In [134]:
x = df_final.drop('Salary', axis= 1)
y = df_final[['Salary']]

from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(x,y, train_size=0.7 ,random_state=125)

In [136]:
train=pd.concat([xtrain,ytrain],axis=1)
train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League,Division,NewLeague,Salary
248,127,32,4.0,14,25,12,19,8396.0,2317,208.75,1048.0,918.75,714.75,167,18,6,1,1,1,6.214608
117,151,41,4.0,26,21,19,2,288.0,68,9.00,45.0,39.00,35.00,28,56,2,0,1,0,4.553877
1,479,130,18.0,66,72,76,3,1624.0,457,63.00,224.0,266.00,263.00,636,82,14,0,1,0,6.173786
3,321,87,10.0,39,42,30,2,396.0,101,12.00,48.0,46.00,33.00,636,40,4,1,0,1,4.516339
37,457,101,14.0,42,63,22,17,6521.0,1767,208.75,1003.0,918.75,619.00,389,39,4,0,1,0,6.774224


In [138]:
test=pd.concat([xtest,ytest],axis=1)
test.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League,Division,NewLeague,Salary
135,591,184,20.0,83,79,38,5,1689.0,462,40.00,219.0,195.00,82.00,303,12,5,1,1,1,6.445720
36,526,146,13.0,71,70,84,6,2648.0,715,77.00,352.0,342.00,289.00,303,9,9,1,1,1,6.703188
152,491,141,11.0,77,47,37,15,4291.0,1240,84.00,615.0,430.00,340.00,239,8,2,0,0,0,6.263398
109,663,200,29.0,108,121,32,4,1447.0,404,57.00,210.0,222.00,68.00,241,8,6,0,0,0,5.521461
49,585,139,31.0,93,94,62,17,7546.0,1982,208.75,1085.5,918.75,714.75,0,0,0,0,0,0,6.856462


In [140]:
# cor=train.corr()
# cor.style.applymap(lambda x: 'background-color : yellow' if x > 0.7 and x!=1 else '')

In [142]:
# cor2=test.corr()
# cor2.style.applymap(lambda x: 'background-color : yellow' if x > 0.7 and x!=1 else '')

In [144]:
train.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League,Division,NewLeague,Salary
248,127,32,4.0,14,25,12,19,8396.0,2317,208.75,1048.0,918.75,714.75,167,18,6,1,1,1,6.214608
117,151,41,4.0,26,21,19,2,288.0,68,9.00,45.0,39.00,35.00,28,56,2,0,1,0,4.553877
1,479,130,18.0,66,72,76,3,1624.0,457,63.00,224.0,266.00,263.00,636,82,14,0,1,0,6.173786
3,321,87,10.0,39,42,30,2,396.0,101,12.00,48.0,46.00,33.00,636,40,4,1,0,1,4.516339
37,457,101,14.0,42,63,22,17,6521.0,1767,208.75,1003.0,918.75,619.00,389,39,4,0,1,0,6.774224


# ada boost

In [146]:
from sklearn.ensemble import AdaBoostRegressor  


### getting top features

In [150]:
xtrain.columns

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists',
       'Errors', 'League', 'Division', 'NewLeague'],
      dtype='object')

In [152]:
# Re-fit the model with the best parameters
final_mod_ADA = AdaBoostRegressor()
final_mod_ADA.fit(xtrain, ytrain)

AdaBoostRegressor()

In [154]:
xtrain.columns

Index(['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat',
       'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists',
       'Errors', 'League', 'Division', 'NewLeague'],
      dtype='object')

In [156]:
importances = final_mod_ADA.feature_importances_

In [158]:
for feature, importance in zip(xtrain.columns, importances):
    print(f"{feature}: {importance}")


AtBat: 0.05551226467991762
Hits: 0.02144124014289653
HmRun: 0.09302094633722346
Runs: 0.03251732776826918
RBI: 0.05192799998375475
Walks: 0.05168358496834352
Years: 0.020450913818774487
CAtBat: 0.12298360618050846
CHits: 0.08895247102306783
CHmRun: 0.0511910074137596
CRuns: 0.08390112398762364
CRBI: 0.07451354767555968
CWalks: 0.13838003729150467
PutOuts: 0.048474300663037054
Assists: 0.026283501945612152
Errors: 0.011294910892200302
League: 0.0013430782176627983
Division: 0.026128137010284415
NewLeague: 0.0


In [160]:
sorted_features = sorted(zip(xtrain.columns, importances), key=lambda x: x[1], reverse=True)

top_8_features = sorted_features[:8]

feature_importance = [feature for feature, importance in top_8_features]

In [162]:
feature_importance

['CWalks', 'CAtBat', 'HmRun', 'CHits', 'CRuns', 'CRBI', 'AtBat', 'RBI']

In [164]:
xtrain=xtrain[feature_importance]

In [166]:
xtrain.columns

Index(['CWalks', 'CAtBat', 'HmRun', 'CHits', 'CRuns', 'CRBI', 'AtBat', 'RBI'], dtype='object')

### model on top features

In [168]:
model_parameters = {'n_estimators': [30, 50, 100],  # number of Tree (Weak Model)
                    'learning_rate': [0.2, 0.6, 0.4, 0.8,1]}

In [170]:
abc = AdaBoostRegressor() 
gscv_ADA = GridSearchCV(estimator=abc, 
                    param_grid=model_parameters, 
                    cv=10, 
                    verbose=1, ### message how many task done at backend
                    n_jobs=-1, ## parallel processing used entire processor
                    scoring='accuracy') ## accuracy evaluation

gscv_ADA.fit(xtrain, ytrain)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


GridSearchCV(cv=10, estimator=AdaBoostRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [0.2, 0.6, 0.4, 0.8, 1],
                         'n_estimators': [30, 50, 100]},
             scoring='accuracy', verbose=1)

In [172]:
print('The best parameter are -', gscv_ADA.best_params_)

The best parameter are - {'learning_rate': 0.2, 'n_estimators': 30}


In [174]:
# Re-fit the model with the best parameters
final_mod_ADA2 = AdaBoostRegressor(**gscv_ADA.best_params_)
final_mod_ADA2.fit(xtrain, ytrain)

AdaBoostRegressor(learning_rate=0.2, n_estimators=30)

In [176]:
xtrain.columns

Index(['CWalks', 'CAtBat', 'HmRun', 'CHits', 'CRuns', 'CRBI', 'AtBat', 'RBI'], dtype='object')

In [178]:
xtrain.head()

,CWalks,CAtBat,HmRun,CHits,CRuns,CRBI,AtBat,RBI
248,714.75,8396.0,4.0,2317,1048.0,918.75,127,25
117,35.00,288.0,4.0,68,45.0,39.00,151,21
1,263.00,1624.0,18.0,457,224.0,266.00,479,72
3,33.00,396.0,10.0,101,48.0,46.00,321,42
37,619.00,6521.0,14.0,1767,1003.0,918.75,457,63


In [180]:
ytrain.head()

,Salary
248,6.214608
117,4.553877
1,6.173786
3,4.516339
37,6.774224


In [189]:
train=pd.concat([xtrain,ytrain],axis=1)

In [191]:
# train_pred = final_mod_ADA.predict(xtrain)

In [193]:
train['prediction_ada']=final_mod_ADA2.predict(xtrain)

In [195]:
train.head()

,CWalks,CAtBat,HmRun,CHits,CRuns,CRBI,AtBat,RBI,Salary,prediction_ada
248,714.75,8396.0,4.0,2317,1048.0,918.75,127,25,6.214608,6.212698
117,35.00,288.0,4.0,68,45.0,39.00,151,21,4.553877,4.709725
1,263.00,1624.0,18.0,457,224.0,266.00,479,72,6.173786,6.348598
3,33.00,396.0,10.0,101,48.0,46.00,321,42,4.516339,4.709725
37,619.00,6521.0,14.0,1767,1003.0,918.75,457,63,6.774224,6.994018


In [197]:
train.columns

Index(['CWalks', 'CAtBat', 'HmRun', 'CHits', 'CRuns', 'CRBI', 'AtBat', 'RBI',
       'Salary', 'prediction_ada'],
      dtype='object')

In [199]:
from sklearn.metrics import r2_score

In [201]:
r2 = r2_score(train.Salary,train.prediction_ada)
print('R2 score for model Performance on Train : ', np.round(r2,2)*100)

R2 score for model Performance on Train :  85.0


In [203]:
test.head()

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,PutOuts,Assists,Errors,League,Division,NewLeague,Salary
135,591,184,20.0,83,79,38,5,1689.0,462,40.00,219.0,195.00,82.00,303,12,5,1,1,1,6.445720
36,526,146,13.0,71,70,84,6,2648.0,715,77.00,352.0,342.00,289.00,303,9,9,1,1,1,6.703188
152,491,141,11.0,77,47,37,15,4291.0,1240,84.00,615.0,430.00,340.00,239,8,2,0,0,0,6.263398
109,663,200,29.0,108,121,32,4,1447.0,404,57.00,210.0,222.00,68.00,241,8,6,0,0,0,5.521461
49,585,139,31.0,93,94,62,17,7546.0,1982,208.75,1085.5,918.75,714.75,0,0,0,0,0,0,6.856462


In [205]:
xtest=xtest[feature_importance]

In [209]:
xtest.columns

Index(['CWalks', 'CAtBat', 'HmRun', 'CHits', 'CRuns', 'CRBI', 'AtBat', 'RBI'], dtype='object')

In [211]:
test=pd.concat([xtest,ytest],axis=1)

In [213]:
test.head()

,CWalks,CAtBat,HmRun,CHits,CRuns,CRBI,AtBat,RBI,Salary
135,82.00,1689.0,20.0,462,219.0,195.00,591,79,6.445720
36,289.00,2648.0,13.0,715,352.0,342.00,526,70,6.703188
152,340.00,4291.0,11.0,1240,615.0,430.00,491,47,6.263398
109,68.00,1447.0,29.0,404,210.0,222.00,663,121,5.521461
49,714.75,7546.0,31.0,1982,1085.5,918.75,585,94,6.856462


In [215]:
test['prediction_ada']=final_mod_ADA2.predict(xtest)

In [217]:
r2 = r2_score(test.Salary,test.prediction_ada)
print('R2 score for model Performance on Test : ', np.round(r2,2)*100)

R2 score for model Performance on Test :  74.0


In [219]:
test.head()

,CWalks,CAtBat,HmRun,CHits,CRuns,CRBI,AtBat,RBI,Salary,prediction_ada
135,82.00,1689.0,20.0,462,219.0,195.00,591,79,6.445720,6.026063
36,289.00,2648.0,13.0,715,352.0,342.00,526,70,6.703188,6.679077
152,340.00,4291.0,11.0,1240,615.0,430.00,491,47,6.263398,6.704144
109,68.00,1447.0,29.0,404,210.0,222.00,663,121,5.521461,6.008781
49,714.75,7546.0,31.0,1982,1085.5,918.75,585,94,6.856462,7.052249


In [223]:
xtrain.columns

Index(['CWalks', 'CAtBat', 'HmRun', 'CHits', 'CRuns', 'CRBI', 'AtBat', 'RBI'], dtype='object')

In [221]:
import pickle

pickle.dump(final_mod_ADA2, open(r"C:\Users\91909\Desktop\PROJECTS\baseball regression project\baseball_ada.pkl",'wb'))

                                
                                
                                # "C:\Users\91909\Desktop\baseball regression project\baseball.pkl",'wb'))  

In [87]:
pip install mysql-connector-python


   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
    --------------------------------------- 0.3/16.4 MB ? eta -:--:--
   - -------------------------------------- 0.5/16.4 MB 1.4 MB/s eta 0:00:12
   - -------------------------------------- 0.8/16.4 MB 1.2 MB/s eta 0:00:14
   -- ------------------------------------- 1.0/16.4 MB 1.3 MB/s eta 0:00:13
   --- ------------------------------------ 1.6/16.4 MB 1.4 MB/s eta 0:00:11
   ---- ----------------------------------- 1.8/16.4 MB 1.5 MB/s eta 0:00:10
   ----- ---------------------------------- 2.1/16.4 MB 1.3 MB/s eta 0:00:12
   ----- ---------------------------------- 2.4/16.4 MB 1.4 MB/s eta 0:00:11
   ------ --------------------------------- 2.6/16.4 MB 1.4 MB/s eta 0:00:10
   ------- -------------------------------- 2.9/16.4 MB 1.4 MB/s eta 0:00:10
   -------- ------------------------------- 3.4/16.4 MB 1.4 MB/s eta 0:00:10
   --------- -------